In [1]:
import pandas as pd 
import requests
from pathlib import Path
from urllib.parse import urlsplit
import time

In [2]:
# Step 1: Load spreadsheet and initialize result file
df = pd.read_excel('data.xlsx', engine='openpyxl')
RESULTS = []
RESULTS.append(['Document ID', 
                'Total Links',
                'Link',
                'Download Path',
                'Status',
                'Code'])
RESULTS_FILE = 'results.csv'

In [3]:
# Step 2: Specify the column with the links
links_column = 'Attachment Files'
id_column = 'Document ID'

In [4]:
# Step 3: Create a folder to save the downloaded files
download_folder = Path('downloads')
download_folder.mkdir(exist_ok=True)

In [5]:
# Step 4: Function to get the file extension from the URL or headers
def get_file_extension(url, response):
    # Try to extract from the URL
    path = urlsplit(url).path
    extension = Path(path).suffix
    if extension:
        return extension
    # Fall back to the content-type header
    content_type = response.headers.get('Content-Type')
    if content_type:
        return f'.{content_type.split("/")[-1]}'
    return '.bin'  # Default to .bin if no extension is found

In [ ]:
# Step 5: Download files from the links
for index, links in enumerate(df[links_column]):
    if pd.notna(links):
        link_list = [link.strip() for link in links.split(',')]  # Split the links and remove any leading/trailing whitespace
        for link_index, link in enumerate(link_list):
            n_docs = len(link_list)
            try:
                response = requests.get(link)
                if response.status_code == 200:
                    extension = get_file_extension(link, response)
                    name = df[id_column].iloc[index]
                    file_name = download_folder / f'{name}_{link_index + 1}{extension}'
                    with open(file_name, 'wb') as f:
                        f.write(response.content)
                    print(f'Downloaded: {file_name}')
                    status = 'Success'
                    RESULTS.append([df[id_column].iloc[index], n_docs, link_index, file_name, status, ''])
                else:
                    print(f'Failed to download {link}: Status code {response.status_code}')
                    status = 'Failed'
                    RESULTS.append([df[id_column].iloc[index], n_docs, link_index, file_name, status, response.status_code])
            except Exception as e:
                print(f'Error downloading {link}: {e}')
                status = 'Error'
                RESULTS.append([df[id_column].iloc[index], n_docs, link_index, file_name, status, ''])

            time.sleep(10)
    else:
        n_docs = 0
        print(f'Skipping row {index}: No link found')
        status = 'Skipped'
        RESULTS.append([df[id_column].iloc[index], n_docs, link_index, file_name, status, response.status_code])


In [14]:
# Step 6: Convert scrape results to data frame to verify success
df_results = pd.DataFrame(RESULTS, columns=RESULTS[0])
df_results.to_csv(RESULTS_FILE, index=False)